In [6]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os
from ray.tune.registry import register_env
from ray.rllib.algorithms.ppo import PPOConfig
import warnings
import ray
import numpy as np
from tqdm import trange
warnings.filterwarnings('ignore')


In [8]:
MAX_EPS = 100
agent_name = 'Blue'

def wrap(env):
    return RLlibWrapper(agent_name="Blue", env=env)

def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

register_env(name="CybORG", env_creator=env_creator)

config = PPOConfig().resources(num_gpus=1).environment(env = 'smac').rollouts(num_rollout_workers=1)

trainer = config.build("CybORG")
path = 'results/APPO/explore/PPO_CybORG_5000a_00000_0_exploration_config=type_RE3_embeds_dim_128_beta_schedule_constant_sub_exploration_type_StochasticSampling_2022-12-05_22-01-04/checkpoint_004902'
path = 'results/APPO/explore/PPO_CybORG_5000a_00001_1_exploration_config=type_StochasticSampling,fcnet_activation=tanh,fcnet_hiddens=256_256_2022-12-06_02-59-07/checkpoint_004902'
trainer.load_checkpoint(path)

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
obs = []
#print(f'using CybORG v{cyborg_version}, {scenario}\n')
for red_agent in [B_lineAgent]:#, RedMeanderAgent]:

    cyborg = CybORG(path, 'sim', agents={'Red': red_agent})
    wrapped_cyborg = wrap(cyborg)

    observation = wrapped_cyborg.reset()
    obs.append(observation)
    # observation = cyborg.reset().observation

    action_space = wrapped_cyborg.get_action_space(agent_name)
    # action_space = cyborg.get_action_space(agent_name)
    total_reward = []
    actions = []
    for i in trange(MAX_EPS):
        r = []
        #a = []
        # cyborg.env.env.tracker.render()
        for j in range(100):
            action = trainer.compute_single_action(observation, explore=False)
            #action_vec = np.zeros(145)
            #action_vec[int(action)] = 1
            #action = agent.get_action(observation, action_space)
            observation, rew, done, info = wrapped_cyborg.step(action)
            obs.append(observation)
            #actions.append(action_vec)
            # result = cyborg.step(agent_name, action)
            r.append(rew)
            # r.append(result.reward)
           # a.append((str(cyborg.get_last_action('Blue')), str(cyborg.get_last_action('Red'))))
        total_reward.append(sum(r))
        # observation = cyborg.reset().observation
        observation = wrapped_cyborg.reset()
    print(f'Average reward for red agent {red_agent.__name__} and steps {100} is: {mean(total_reward):.1f} with a standard deviation of {stdev(total_reward):.1f}')
    #return mean(total_reward), np.mean(np.array(obs), axis=0),  np.mean(np.array(actions), axis=0)

(RolloutWorker pid=378) 2022-12-06 09:56:35,405	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-12-06 09:56:39,345	WARNING util.py:66 -- Install gputil for GPU system monitoring.
100%|██████████| 100/100 [01:32<00:00,  1.08it/s]

Average reward for red agent B_lineAgent and steps 100 is: -24.5 with a standard deviation of 16.6
